In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
import pickle
import mne
import re

from mne.channels import make_1020_channel_selections
from mne.event import define_target_events
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm


from sklearn.model_selection import KFold, train_test_split, cross_validate
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import warnings
warnings.filterwarnings("ignore")


from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

class_names = ['1','2','3']

In [2]:
os.chdir('/data/castrogaray-j/CH_ERDS')
files = os.listdir()
# pkl_files = sorted([file for file in files if file.startswith('raw') and file.endswith('.pkl')])
# subjects = set([file[4:6] for file in pkl_files])
# #pkl_files
# len(pkl_files)
# int(pkl_files[9][8:10])
print(len(files))
print(files[0])
print(files[0][8:10], files[0][17], files[0][33:36])



403276
subject_18_class_1_label_0_event_165_channel_P2.png
18 1 165


In [3]:
def get_tags(file):
    

    pattern = r"subject_(\d+)_class_(\d+)_label_\d+_event_(\d+)_channel_"
    subject, class_tag, event = re.findall(pattern, file)[0]
    

    

    return  class_tag, subject, int(event) +1
get_tags(files[0])    

('1', '18', 166)

In [4]:
pattern = r"subject_(\d+)_class_(\d+)_label_\d+_event_(\d+)_channel_"
matches = re.findall(pattern, files[0])

for match in matches:
    subject, class_, event = match
    print(f"Subject: {subject}, Class: {class_}, Event: {event}")

Subject: 18, Class: 1, Event: 165


In [5]:
with open("/home/castrogaray-j/U_Winnipeg_OneDrive/Adv_Machine_learning/BCIT_Performance/ICA_Features/5s_ICA_features_labels.pkl", 'rb') as file:
        df = pickle.load(file)

df = df.iloc[:,-8:]
df.rename(columns={'condition': 'class'}, inplace=True)
df

,event,reaction times,subject,class,label,drop 10,drop 20,3 classes
0,1,1.676758,1,1,0,0.0,0.0,0.0
1,1,1.676758,1,1,0,0.0,0.0,0.0
2,1,1.676758,1,1,0,0.0,0.0,0.0
3,1,1.676758,1,1,0,0.0,0.0,0.0
4,1,1.676758,1,1,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
47205373,165,2.080078,21,3,1,1.0,1.0,2.0
47205374,165,2.080078,21,3,1,1.0,1.0,2.0
47205375,165,2.080078,21,3,1,1.0,1.0,2.0
47205376,165,2.080078,21,3,1,1.0,1.0,2.0


In [6]:
df1 = df.drop_duplicates(subset=['class', 'subject', 'event'])
df1.reset_index(drop=True, inplace=True)
df_files = pd.DataFrame(files)
df_files

,0
0,subject_18_class_1_label_0_event_165_channel_P...
1,subject_17_class_3_label_0_event_143_channel_P...
2,subject_17_class_3_label_0_event_147_channel_F...
3,subject_19_class_3_label_0_event_14_channel_P9...
4,subject_17_class_1_label_0_event_80_channel_P6...
...,...
403271,subject_17_class_3_label_1_event_5_channel_AF3...
403272,subject_18_class_2_label_1_event_28_channel_CP...
403273,subject_1_class_2_label_1_event_107_channel_F8...
403274,subject_18_class_1_label_1_event_6_channel_Iz.png


In [7]:

# Aplicar la función get_tags a cada archivo en df_files
#for i in range(0, len(df_files)):
    #df_files.loc[i, 'class'], df_files.loc[i, 'subject'], df_files.loc[i, 'event'] = get_tags(df_files[0][i])
df_files[['class', 'subject', 'event']] = df_files[0].apply(lambda x: pd.Series(get_tags(x)))

# Convertir las columnas a tipo int para que coincidan con df1
df_files['class'] = df_files['class'].astype(int)
df_files['subject'] = df_files['subject'].astype(int)
df_files['event'] = df_files['event'].astype(int)


df_files


,0,class,subject,event
0,subject_18_class_1_label_0_event_165_channel_P...,1,18,166
1,subject_17_class_3_label_0_event_143_channel_P...,3,17,144
2,subject_17_class_3_label_0_event_147_channel_F...,3,17,148
3,subject_19_class_3_label_0_event_14_channel_P9...,3,19,15
4,subject_17_class_1_label_0_event_80_channel_P6...,1,17,81
...,...,...,...,...
403271,subject_17_class_3_label_1_event_5_channel_AF3...,3,17,6
403272,subject_18_class_2_label_1_event_28_channel_CP...,2,18,29
403273,subject_1_class_2_label_1_event_107_channel_F8...,2,1,108
403274,subject_18_class_1_label_1_event_6_channel_Iz.png,1,18,7


In [8]:
df_joined = df1.merge(df_files, on=['class', 'subject', 'event'], how='right')
df_joined.rename(columns={0: 'path'}, inplace=True)
df_joined['path'] = '/data/castrogaray-j/CH_ERDS/' + df_joined['path'].astype(str)
df_joined = df_joined.sort_values(by=['subject', 'class', 'event', 'path']).reset_index(drop=True)
df_joined

,event,reaction times,subject,class,label,drop 10,drop 20,3 classes,path
0,1,1.676758,1,1,0.0,0.0,0.0,0.0,/data/castrogaray-j/CH_ERDS/subject_1_class_1_...
1,1,1.676758,1,1,0.0,0.0,0.0,0.0,/data/castrogaray-j/CH_ERDS/subject_1_class_1_...
2,1,1.676758,1,1,0.0,0.0,0.0,0.0,/data/castrogaray-j/CH_ERDS/subject_1_class_1_...
3,1,1.676758,1,1,0.0,0.0,0.0,0.0,/data/castrogaray-j/CH_ERDS/subject_1_class_1_...
4,1,1.676758,1,1,0.0,0.0,0.0,0.0,/data/castrogaray-j/CH_ERDS/subject_1_class_1_...
...,...,...,...,...,...,...,...,...,...
403271,161,2.472656,21,3,1.0,1.0,1.0,2.0,/data/castrogaray-j/CH_ERDS/subject_21_class_3...
403272,161,2.472656,21,3,1.0,1.0,1.0,2.0,/data/castrogaray-j/CH_ERDS/subject_21_class_3...
403273,161,2.472656,21,3,1.0,1.0,1.0,2.0,/data/castrogaray-j/CH_ERDS/subject_21_class_3...
403274,161,2.472656,21,3,1.0,1.0,1.0,2.0,/data/castrogaray-j/CH_ERDS/subject_21_class_3...


In [10]:
df_joined.to_csv('/home/castrogaray-j/U_Winnipeg_OneDrive/Digital_image_processing/BCIT_spectrogram/CH_ERDS_paths_with_labels.csv', index=False)

In [9]:
df_joined.shape[0]/64

6301.1875